In [6]:
import scipy.stats
from sklearn.datasets import load_digits

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [7]:
data = load_digits()
train_X, test_X, train_y, test_y = train_test_split(data.data, data.target, test_size=0.2, random_state=42)

In [8]:
model_param_set_grid = {
    LogisticRegression(): {
       "C": [10 ** i for i in range(-5, 5)],
        "random_state": [42]
    },
    LinearSVC(): {
       "C": [10 ** i for i in range(-5, 5)],
        "multi_class": ["ovr", "crammer_singer"],
        "random_state": [42]
    },
    SVC(): {
        "kernel" : ["linear", "poly", "rbf", "sigmoid"],
       "C": [10 ** i for i in range(-5, 5)],
        "decision_function_shape" : ["ovr", "ovo"],
        "random_state": [42]
    },
    DecisionTreeClassifier(): {
       "max_depth": [i for i in range(1, 20)]
    },
    RandomForestClassifier(): {
       "n_estimators": [i for i in range(10, 20)],
        "max_depth": [i for i in range(1, 10)]
    },
    KNeighborsClassifier(): {
       "n_neighbors": [i for i in range(1, 10)]
    }
}

In [9]:
model_param_set_random = {
    LogisticRegression(): {
       "C": scipy.stats.uniform(0.00001, 1000),
        "random_state": scipy.stats.randint(0, 100)
    },
    LinearSVC(): {
       "C": scipy.stats.uniform(0.00001, 1000),
        "multi_class": ["ovr", "crammer_singer"],
        "random_state": scipy.stats.randint(0, 100)
    },
    SVC(): {
        "kernel" : ["linear", "poly", "rbf", "sigmoid"],
       "C": scipy.stats.uniform(0.00001, 1000),
        "decision_function_shape" : ["ovr", "ovo"],
        "random_state": scipy.stats.randint(0, 100)
    },
    DecisionTreeClassifier(): {
       "max_depth": scipy.stats.randint(1, 20)
    },
    RandomForestClassifier(): {
       "n_estimators": scipy.stats.randint(10, 100),
        "max_depth": scipy.stats.randint(1, 20)
    },
    KNeighborsClassifier(): {
       "n_neighbors": scipy.stats.randint(1, 20)
    }
}

In [10]:
max_score = 0
best_model = None
best_param = None

for model, param in model_param_set_grid.items():
    clf = GridSearchCV(model, param)
    clf.fit(train_X, train_y)
    pred_y = clf.predict(test_X)
    score = f1_score(test_y, pred_y, average="micro")
    if max_score < score:
        max_score = score
        best_model = model.__class__.__name__
        best_param = clf.best_params_
        
for model, param in model_param_set_random.items():
    clf = RandomizedSearchCV(model, param)
    clf.fit(train_X, train_y)
    pred_y = clf.predict(test_X)
    score = f1_score(test_y, pred_y, average="micro")
    if max_score < score:
        max_score = score
        best_model = model.__class__.__name__
        best_param = clf.best_params_
        

print("Model: {}, ¥n Param: {}".format(best_model, best_param))
print(max_score)

Model: SVC, ¥n Param: {'C': 1e-05, 'decision_function_shape': 'ovr', 'kernel': 'poly', 'random_state': 42}
0.9888888888888889
